In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json
import random

In [2]:
train_file = '/data/sulixin/research/contest/pytorch-pretrained-BERT/data/datasets/squadv1/train-v1.1.json'
dev_file = '/data/sulixin/research/contest/pytorch-pretrained-BERT/data/datasets/squadv1/dev-v1.1.json'

In [3]:
train = json.load(open(train_file))
dev = json.load(open(dev_file))

In [4]:
train['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [7]:
def count_p(data):
    print( sum([len(d['paragraphs']) for d in data]))
    
def count_a(data):
    print( sum([ sum( [len(p['qas']) for p in d['paragraphs']]) for d in data]))
count_p(train['data'] + dev['data'])
count_a(train['data'] + dev['data'])

20963
98169


In [5]:
train['data'][0]['paragraphs'][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [6]:
def iter_para(data):
    for w in data:
        for p in w['paragraphs']:
            yield p

def word_len(x):
    return len(x.split())

def avg(li):
    return sum(li) * 1.0 / len(li)

def avg_answer_len(c):
    return avg( [word_len(qa['answers'][0]['text']) for qa in c['qas'] ] )

def avg_query_len(c):
    return avg( [word_len(qa['question']) for qa in c['qas'] ] )

def qa_cnt(c):
    return len(c['qas'])
    
def my_stat(data):
    rv = {'para_len': [word_len(x['context']) for x in iter_para(data)],\
          'ques_len': [avg_query_len(x) for x in iter_para(data)],\
          'answer_len': [avg_answer_len(x) for x in iter_para(data)],\
          'qa_cnt': [qa_cnt(x) for x in iter_para(data)]
         }
    return pd.DataFrame(rv).describe()

In [7]:
my_stat(train['data'])

,para_len,ques_len,answer_len,qa_cnt
count,18896.000000,18896.000000,18896.000000,18896.000000
mean,116.630451,10.094799,3.186379,4.635849
std,49.708795,2.457499,2.479863,1.681008
min,20.000000,2.000000,1.000000,1.000000
25%,87.000000,8.400000,1.800000,4.000000
50%,107.000000,9.800000,2.400000,5.000000
75%,139.000000,11.500000,3.600000,5.000000
max,653.000000,29.000000,24.000000,25.000000


In [8]:
my_stat(dev['data'])

,para_len,ques_len,answer_len,qa_cnt
count,2067.000000,2067.000000,2067.000000,2067.000000
mean,122.776971,10.207862,3.102438,5.113691
std,54.849306,2.460745,2.161528,2.339761
min,22.000000,3.600000,1.000000,1.000000
25%,90.000000,8.600000,1.800000,4.000000
50%,111.000000,10.000000,2.400000,5.000000
75%,144.000000,11.600000,3.500000,5.000000
max,629.000000,23.200000,20.750000,30.000000
